# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
import datetime
from operator import attrgetter
from itertools import product
import itertools
import pickle

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [2]:
# importeren data
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [3]:
# aanpassen van de data (NaN -> 0 en x -> 1)
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [4]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven
data = pd.date_range("2020-01-06", periods=4*10*7, freq='D')

columns = ['Datum', 'Sprint', 'Weeknummer', 'DagRooster', 'Werkdag']

dfKalender = pd.DataFrame(columns=columns)
dfKalender['Datum'] = data
dfKalender = dfKalender.fillna(0)
dfKalender.replace("0", False, inplace=True)

dfKalender['Werkdag'] = dfKalender['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

In [5]:
dfKalender = dfKalender[dfKalender['Werkdag']]
dfKalender = dfKalender.reset_index(drop=True)

In [6]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [7]:
def OpdrachtUitvoerenPersoonBool(eisen, persoon):
    for i in rangeSkills:
        result = (int(eisen[i-1]) <= 
                  int(persoon[i-1]))
        if result == False:
            break
    return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug en het surplus.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [8]:
def PersonenSamenvoegen(persoon1, persoon2 = [0]*13, persoon3 = [0]*13, persoon4 = [0]*13, persoon5 = [0]*13, persoon6 = [0]*13):
    combinedSkill = []
    surplus = 0 
    for i in rangeSkills:
        combinedSkill.append(int(1) if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
        surplus = surplus + (persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] -1 
                         if persoon1[i-1]+persoon2[i-1]+persoon3[i-1]+persoon4[i-1]+persoon5[i-1]+persoon6[i-1] > 0 else int(0))
    combinedSkill.append(surplus)
    return combinedSkill

In [9]:
# het selecteren van alleen de tasks van sprint 1 en het dataframe maken met de columnnames en het resetten van de index
dfProjectTasks = dfProjectTasks[dfProjectTasks['Task'] != 0]
dfSprint1 = pd.DataFrame(dfProjectTasks[dfProjectTasks['Task'].str.contains('S1')], 
                         columns= list(dfProjectTasks)).reset_index(drop=True)


In [10]:
# het maken van de kolom crew en het aanpassen van het typen naar object zodat er een list of array kan worden toegevoegd aan die kolom.
dfSprint1.loc[:,'Crew']= [20]*len(dfSprint1)
dfSprint1['Crew'] = dfSprint1['Crew'].astype(object)

In [11]:
# # het verzamelen van data van alle gezamelijke skills in staan tot aan 7 personen

# dfCrewCombined = []
# for i in range(0, len(dfCrew)): 
#     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills])
#     t.append(i)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(20)
#     t.append(1)
    
#     dfCrewCombined.append(t)
    
#     for j in range(0, len(dfCrew)): # combinaties met twee mensen
#         if i==j:
#             break
#         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills])
#         t.append(i)
#         t.append(j)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(20)
#         t.append(2)
        
#         dfCrewCombined.append(t)

#         for k in range(0, len(dfCrew)):
#             if i==k or k==j:
#                 break
        
#             t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills], dfCrew.iloc[k,rangeSkills])
#             t.append(i)
#             t.append(j)
#             t.append(k)
#             t.append(20)
#             t.append(20)
#             t.append(20)
#             t.append(3)
            
#             dfCrewCombined.append(t)
            
#             for l in range(0, len(dfCrew)):
#                 if l == k or l == i or l == j :
#                     break
#                 t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                         dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills])
#                 t.append(i)
#                 t.append(j)
#                 t.append(k)
#                 t.append(l)
#                 t.append(20)
#                 t.append(20)
#                 t.append(4)
#                 dfCrewCombined.append(t)

#                 for m in range(0, len(dfCrew)):
#                     if m==l or m==k or m==j or m==i:
#                         break
#                     t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                             dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                             dfCrew.iloc[m,rangeSkills])
#                     t.append(i)
#                     t.append(j)
#                     t.append(k)
#                     t.append(l)
#                     t.append(m)
#                     t.append(20)
#                     t.append(5)
#                     dfCrewCombined.append(t)
                    
#                     for n in range(0, len(dfCrew)):
#                         if n==m or n==l or n==k or n==j or n==i:
#                             break
#                         t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
#                                                 dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
#                                                 dfCrew.iloc[m,rangeSkills], dfCrew.iloc[n,rangeSkills])
#                         t.append(i)
#                         t.append(j)
#                         t.append(k)
#                         t.append(l)
#                         t.append(m)
#                         t.append(n)
#                         t.append(6)
#                         dfCrewCombined.append(t)


                    
       

het dataframe dfCrewCombined is een dataframe met alle skills van de medewerkers gezamelijk. Zodra er bij een van de personen '20' staat, betekent dit dat er geen persoon voor die plek is toegevoegd. in het dataframe staan combinaties met twee personen tot zeven ('vo) personen.


In [12]:
# # het maken van het dataframe dfCrewCombined en het wegschrijven in csv.
# ar = np.array(dfCrewCombined, dtype=np.int16) 
# dfCrewCombined = pd.DataFrame(ar, columns= ['Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'Skill6', 'Skill7', 'Skill8', 
#                                 'Skill9', 'Skill10', 'Skill11', 'Skill12', 'Skill13', 'Surplus', 'persoon1', 'persoon2', 
#                                 'persoon3', 'persoon4', 'persoon5', 'persoon6', 'Aantalpersonen'])

# # creeren van de kolom 'sum' zodat we weten hoeveel skill de crew gecombineerd heeft
# dfCrewCombined['sum'] = dfCrewCombined.iloc[:,0:13].sum(axis = 1) 

# # sorteren op de kolommen sum en surplus (overschot aan skills) zodat er zo min mogelijk overbodige skills worden gebruikt 
# dfCrewCombined = dfCrewCombined.sort_values(['sum', 'Surplus'], ascending= [True, True])
# dfCrewCombined = dfCrewCombined.reset_index(drop=True)

# dfCrewCombined.to_csv('CrewCombined.csv', encoding='utf-8', index=False) # schrijven naar csv

In [13]:
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

In [14]:
class ProjectUren:
    def __init__(self, project, uren):
        self.project = project
        self.uren = uren
        
class RoosterDag:
    
    def __init__(self, crew, datum, urenTeGaan=7, projectUren=['x', 'x']):
        self.crew = crew
        self.datum = datum
        self.urenTeGaan = urenTeGaan
        self.projectUren = ProjectUren(*projectUren)        

In [15]:
class MogelijkeCombinaties:
    def __init__(self, dfMogelijkheden, aantalMogelijkheden):
        self.dfMogelijkheden = dfMogelijkheden
        self.aantalMogelijkheden = aantalMogelijkheden

In [16]:
class Rooster:
    def __init__(self, dfRooster):
        self.dfRooster = dfRooster

In [17]:
def Lijstmogelijkheden(crew, duration): # lijst met mogelijke data wordt teruggegeven zodra de lijn van de crew en de duur van de task wordt ingevuld
    lijstmogelijkheden = []
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.urenTeGaan >= duration :
                lijstmogelijkheden.append(dag)
    if len(lijstmogelijkheden) == 0:
        lijstmogelijkheden.append(RoosterDag("geen medewerker",datetime.date(2100,1,1)))
                                  
    return lijstmogelijkheden

In [18]:
# deze zou overbodig zijn
def add_project(*projectUren): # de huidige moet worden mee gegeven en degene die je wilt toevoegen.
    # Hoeveel je er wilt toevoegen doet er niet toe.
    lijst = []
    for project in projectUren:
        if type(project) == list:
            lijst.append(ProjectUren(project[0], project[1]))
        else:
            if project.project != "x":
                lijst.append(project)
    return lijst

In [19]:
# deze zou overbodig zijn.
# vind een Roosterdag met crewlid en datum (dit zou uniek moeten zijn)
def VindRoosterdag(crew, datum):
    for dag in werkdagen:
        if crew == dag.crew:
            if dag.datum == datum: 
                return dag

Voorbeeld voor de functie add_project <br>
t = test2(["S",2]) <br>
t.projectUren = add_project(t.projectUren, ["D", 2]) <br>
<br><br>
for project in t.projectUren: <br>
    print (project.project) <br>
    
Uitkomst: <br>
S<br>
D<br>

In [20]:
# # de eerste dag aanmaken voor iedereen
# werkdagen = []
# for i in range(0, len(dfCrew)):
#     werkdagen.append(RoosterDag(i, datetime.date(2020,1, 1)))

In [21]:
# # code voor het invullen van crew bij een tascrewLijn waarbij nog niet wordt gelet op of mensen al bezet zijn.

# for j in range(0, len(dfSprint1)):
#     inplannen = False
#     i = 0
#     gevondenDatum = False
#     print("j: ",j)
#     eersteDatum = datetime.date(2020,1, 1)
    
#     while inplannen == False:
#         if OpdrachtUitvoerenPersoonBool(dfSprint1.iloc[j,rangeSkills], dfCrewCombined.iloc[i,0:13]): # !TODO min en max mensen
#             print("task met crew gevonden")
#             dfSprint1.at[j,'Crew'] = np.array(dfCrewCombined.iloc[i,14:20])
#             print(dfSprint1.Crew[j])
#             for crewLijn in dfCrewCombined.iloc[i,14:20].tolist():
#                 if crewLijn ==20:
#                     break
                
#                 #if min(Lijstmogelijkheden(crewLijn, dfSprint1.Duration[j]), key=attrgetter('datum')).datum <= eersteDatum:
#                 if VindRoosterdag(crewLijn, eersteDatum).urenTeGaan >= dfSprint1.Duration[j]:
#                     gevondenDatum = True
#                 else:
#                     gevondenDatum = False
#                     break
                
#                 print("datum gevonden ", gevondenDatum)
#             if gevondenDatum:
#                 for crewLijn in dfCrewCombined.iloc[i,14:20].tolist():
#                     if crewLijn ==20:
#                         break
#                     dag = VindRoosterdag(crewLijn, eersteDatum)
#                     print("dag gevonden ", eersteDatum)
#                     print("uren die nog moeten worden gemaakt: ", dag.urenTeGaan)
#                     dag.urenTeGaan = dag.urenTeGaan - dfSprint1.Duration[j]
#                     print(dag)
                    
#                     print(dfSprint1.Task[j], dfSprint1.Duration[j])
#                     if type(dag.projectUren) == list:
#                         dag.projectUren.append(ProjectUren(dfSprint1.Task[j], dfSprint1.Duration[j]))
#                     else:
#                         dag.projectUren = add_project(dag.projectUren, [dfSprint1.Task[j], dfSprint1.Duration[j]])
# #                     werkdagen.append(RoosterDag(k, eersteDatum, 7- dfSprint1.Duration[j] , [dfSprint1.Task[j], dfSprint1.Duration[j]]))
#                 i=0
#                 inplannen = True
#             else:
#                 print("volgende datum vinden")
#                 eersteDatum = eersteDatum + datetime.timedelta(days=1) # misschien wordt dit te vaak gedaan.
#                 for crewLijn in range(0, len(dfCrew)):
#                     werkdagen.append(RoosterDag(crewLijn, eersteDatum))
#         else:
#             i = i + 1


Misschien moeten we aan het begin geen eens moeite doen met hoe we de pauzes indelen. gewoon iemand zeven uur per dag laten werken. en dan de volgende dag iets laten beginnen dat pas kan beginnen als de andere taak af is. dan kunnen we later dat nog optimaliseren.

https://www.daniweb.com/programming/software-development/code/216631/a-list-of-class-objects-python
Goede site om een beetje te kijken hoe je met de lijst van objecten moet werken.

In [22]:
# werkdagen = []
# werkdagen.append(RoosterDag("Robbert", "3-10-1996", 5, ["S1", 2]))
# werkdagen.append(RoosterDag("Robber", "3-10-1996", 4, ["S1", 2]))
# werkdagen.append(RoosterDag("Robert", "3-10-1996", 3, ["S1", 2]))

# werkdagen.append(RoosterDag("Robbert", "2-10-1996", 3, ["S1",3]))

# werkdagen.append(RoosterDag("Robbert", "3-9-1996"))

In [23]:
# werkdagen.sort(key=operator.attrgetter('datum'))

In [24]:
dfSprint1.loc[:,'Mogelijkheden']= [20]*len(dfSprint1)
dfSprint1.loc[:,'AantalMogelijkheden']= [20]*len(dfSprint1)
dfSprint1['Mogelijkheden'] = dfSprint1['Mogelijkheden'].astype(object)
dfSprint1['AantalMogelijkheden'] = dfSprint1['AantalMogelijkheden'].astype(int)
dfSprint1['MaxReqCrew'].replace(0, 6, inplace=True)


In [25]:
def ZoekKamers(eisenKamers):
    lijstkamers = []
    for i in range(0, len(eisenKamers)):
        if list(eisenKamers)[i] == 1:
            if ("****" in list(dfSprint1)[14+i]):
                lijstkamers.append("assembly A")
                lijstkamers.append("assembly B")
            if ("***" in list(dfSprint1)[14+i] and not "****" in list(dfSprint1)[14+i]):
                lijstkamers.append("workshop A")
                lijstkamers.append("workshop B")
                lijstkamers.append("workshop C")
            if ("**" in list(dfSprint1)[14+i] and not "***" in list(dfSprint1)[14+i]):
                lijstkamers.append("workspace A")
                lijstkamers.append("workspace B")
                lijstkamers.append("workspace C")
                
            elif ("4" in list(dfSprint1)[14+i] or "5" in list(dfSprint1)[14+i]):
                lijstkamers.append("office A1")
                lijstkamers.append("office A2")
                lijstkamers.append("office A3")
                lijstkamers.append("office A4")
                lijstkamers.append("office A5")
                lijstkamers.append("office B1")
                lijstkamers.append("office B2")
                lijstkamers.append("office B3")
                lijstkamers.append("office B4")
                lijstkamers.append("office C1")
                lijstkamers.append("office C2")
                lijstkamers.append("office C3")
                lijstkamers.append("office C4")
                
            elif ("technical" in list(dfSprint1)[14+i]):
                lijstkamers.append("technical A")
                lijstkamers.append("technical B")
                
    return lijstkamers

In [26]:
def ZoekOpdrachten(project, MinMaxPersonen):
    crew = []
    for i in range(0, len(dfCrewCombined)):
        if (dfCrewCombined.Aantalpersonen[i] >= MinMaxPersonen[0] and dfCrewCombined.Aantalpersonen[i] <= MinMaxPersonen[1]):
            if (OpdrachtUitvoerenPersoonBool(project, dfCrewCombined.iloc[i,0:13])):
                crew.append(list(dfCrewCombined.iloc[i,14:20]))
    return crew
                        

In [27]:
dfSprint1['Voltooid'] = False
dfSprint1['Eis'] = ""
dfSprint1['Eis']=dfSprint1['Task'].apply(lambda Task: Task[:-1]+'A' if Task.endswith('B') else 'no')
dfSprint1['Moment Voltooid']= ""

In [28]:
for i in range(0,len(dfSprint1)):
    a = ZoekKamers(dfSprint1.iloc[i,14:20])
#     if i!=0:

#         if ((dfSprint1["Eis"][i] != "no") & (dfSprint1["Task"][i-1] == dfSprint1["Eis"][i])):
#             b = ZoekOpdrachten(PersonenSamenvoegen(dfSprint1.iloc[i-1,rangeSkills],dfSprint1.iloc[i,rangeSkills]), dfSprint1.iloc[i,20:22]) # eis is hier verwerkt
#         else:
#             b = ZoekOpdrachten(dfSprint1.iloc[i,rangeSkills], dfSprint1.iloc[i,20:22]) 
#     else:
    b = ZoekOpdrachten(dfSprint1.iloc[i,rangeSkills], dfSprint1.iloc[i,20:22]) 
    dfSprint1.iloc[i,26] = MogelijkeCombinaties(pd.DataFrame(product(["kamer"], b)), len(a) * len(b))
    dfSprint1.iloc[i,27] = len(a) * len(b) # >> is een optie, hoeft op zich niet, misschien wel makkelijker

In [29]:
# import json
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(dfSprint1, f, ensure_ascii=False, indent=4)

In [30]:
# dfSprint1.to_csv('dfSprint1.csv', encoding='utf-8', index=False)

In [31]:
# dfSprint1 = pd.read_csv('dfSprint1.csv')

In [32]:
dfSprint1 = dfSprint1.sort_values(['AantalMogelijkheden'], ascending= [True])
dfSprint1 = dfSprint1.reset_index(drop=True)


In [33]:
# het maken van een dataframe als kalender << deze moet worden aangepast naar de info van hierboven
tijd = pd.date_range("09:00", "17:30", freq="30min").time
tijd = np.delete(tijd, 13, axis=0)
tijd = np.delete(tijd, 6, axis=0)

columns = np.array(dfCrew['Crew members'])

dfRooster = pd.DataFrame(columns=columns)
dfRooster['Tijd'] = tijd
dfRooster = dfRooster.fillna(0)
dfRooster.replace("0", False, inplace=True)

# volgorde veranderen
cols = dfRooster.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfRooster = dfRooster[cols]

In [34]:
a = [7]*(len(dfRooster.iloc[1, :])-1)
a.insert(0,'urenTeGaan')

b = dfRooster.columns.tolist()
new_row = pd.DataFrame(a).T
new_row.columns = b

dfRooster = pd.concat([new_row, dfRooster]).reset_index(drop = True)
StandaardRooster = dfRooster

In [35]:
import copy

for i in range(0,5):
    rooster = copy.deepcopy(StandaardRooster)
    rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist()
    dfKalender.DagRooster[i] = Rooster(rooster)

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
def contains(sub, pri):
    M, N = len(pri), len(sub)
    i, LAST = 0, M-N+1
    while True:
        try:
            found = pri.index(sub[0], i, LAST) # find first elem in sub
        except ValueError:
            return False
        if pri[found:found+N] == sub:
            return [found, found+N-1]
        else:
            i = found+1

In [37]:
def CombinatieLijst(lijst1, lijst2):
    lijstfinal = []
    if ((len(lijst1) == 16) & (len(lijst2) == 16)):
        for i in range(0,len(lijst1)):
            if((lijst1[i]==0) & (lijst2[i] == 0)):
                lijstfinal.append(0)
            else:
                lijstfinal.append(1)
        return lijstfinal
    else:
        Exception("Lengte van de lijst klopt niet")
        print("Lengte van de lijst klopt niet")
        return
    

!TODO kamers zijn nog buitenbeschouwing gelaten, staan ook niet in een dataframe

In [38]:
# def InplannenOpdracht(taskdf, dagIndex): 
# # task moet een rij van het dataframe dfSprint1 zijn en een dag (als index) van een rij in dfKalender (verschil ten opzichten van 2020-1-6)
#     print("Aanroep functie InplannenOpdracht met dagindex: " + str(dagIndex))
  
#     for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
#         print("De crewlijst die wordt uitgeprobeerd: " + str(crewlijst))
#         lijstVrijeUren = [0]*16
#         controle = True
#         for i in crewlijst:
#             if (i == 20):
#                 break
#             if (dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[0,i+1])< taskdf.Duration:
#                 print("De controle van het aantal uur te gaan is False")
#                 controle = False
#                 break
#             print("de datum " + str(dfKalender.iloc[dagIndex,:].Datum) + " voor persoon " +str(i))
#             print("Het aantal uren te gaan voor persoon i " + str(dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[0,i+1]))
#             lijstVrijeUren = CombinatieLijst(dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
        
#         duration = [0]*int(round(taskdf.Duration*2))
#         print("De lijstvrije uren ziet er als uit: " + str(lijstVrijeUren))
#         if (contains(duration, lijstVrijeUren) != False) & controle:
#             rijbegin , rijeind = contains(duration, lijstVrijeUren)
#             rijbegin += 1
#             rijeind += 1
#             print("het begin en het einde van de rij: "+ str(rijbegin) +" "+ str(rijeind))
#             for i in crewlijst:
#                 if i == 20:
#                     break
                
#                 dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[rijbegin:rijeind+1,i+1] = taskdf.Task + " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
#                 dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalender.iloc[dagIndex,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)
            
#             dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Voltooid'] = True
#             print("ingepland" + taskdf.Task)
#             return
    
    

In [39]:
def InplannenOpdracht(taskdf, dagIndex, dfKalender): 
# task moet een rij van het dataframe dfSprint1 zijn en een dag (als index) van een rij in dfKalender (verschil ten opzichten van 2020-1-6)
    print("Aanroep functie InplannenOpdracht met dagindex: " + str(dagIndex))
    tabuList = []
    
    print("Er wordt ene kopie gemaakt van dfKalender")
    dfTemp = pickle.loads(pickle.dumps(dfKalender))
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist():
        print("De crewlijst die wordt uitgeprobeerd: " + str(crewlijst))
        lijstVrijeUren = [0]*16
        controle = True
        dagIndexKopie = dagIndex
        
        freq = 1
        while freq <= int(taskdf.Frequency):
            print("bovenaan de while loop met: "+ taskdf.Task)
            for i in crewlijst:
                print("Kijken naar i: " + str(i))
                if (i == 20):
                    break
                if ((float(dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)):
                    print("De vergeleken waardes: "+ str(dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1])+  " "+ str(task.Duration) )
                    print("De controle van het aantal uur te gaan is False")
                    controle = False
                    break
                print("de datum " + str(dfKalender.iloc[dagIndexKopie,:].Datum) + " voor persoon " +str(i))
                print("Het aantal uren te gaan voor persoon i " + str(dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))
                lijstVrijeUren = CombinatieLijst(dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment
            duration = [0]*int(round(taskdf.Duration*2))

            print("De lijstvrije uren ziet er als uit: " + str(lijstVrijeUren))
            if (contains(duration, lijstVrijeUren) != False) & controle:
                rijbegin , rijeind = contains(duration, lijstVrijeUren)
                rijbegin += 1
                rijeind += 1
                print("het begin en het einde van de rij: "+ str(rijbegin) +" "+ str(rijeind))
                for i in crewlijst:
                    if i == 20:
                        break

                    dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin:rijeind+1,i+1] = taskdf.Task + " freq: " + str(freq)+ " " + str(taskdf.Mogelijkheden.dfMogelijkheden.iloc[crewlijst,0].iloc[0])
                    dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalender.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration)
                
                if (freq == taskdf.Frequency):
                    print("Voltooid " + taskdf.Task)
                    dfSprint1.loc[dfSprint1['Task'] == taskdf.Task, 'Voltooid'] = True
                    return 
                
                print("ingepland" + taskdf.Task + " " + str(freq))
                freq += 1
            else:
                print("dagindex wordt verhoogd naar: " + str(dagIndexKopie+1))
                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
#                 if (freq == 0):
#                     return dfKalender
                if (dagIndexKopie == 5):
                    print("Een kopie wordt teruggezet van dfKalender")
                    dfKalender = pickle.loads(pickle.dumps(dfTemp))
                    break
    return 
    
    

!TODO Frequentie

!TODO opdrachten over twee dagen

In [40]:
dfk = pickle.loads(pickle.dumps(dfKalender))
dfs = pickle.loads(pickle.dumps(dfSprint1))

In [41]:
# dfKalender = pickle.loads(pickle.dumps(dfk))
# dfSprint1 = pickle.loads(pickle.dumps(dfs))

In [42]:
for i in range(0,len(dfSprint1)):
    task = dfSprint1.iloc[i,:]
    dagIndex = 0
    print(task.Task + " met: " + str(task.Voltooid))
    InplannenOpdracht(task,dagIndex, dfKalender)

S1C3T5B met: False
Aanroep functie InplannenOpdracht met dagindex: 0
Er wordt ene kopie gemaakt van dfKalender
De crewlijst die wordt uitgeprobeerd: [5, 20, 20, 20, 20, 20]
bovenaan de while loop met: S1C3T5B
Kijken naar i: 5
de datum 2020-01-06 00:00:00 voor persoon 5
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 20
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 1 4
ingeplandS1C3T5B 1
bovenaan de while loop met: S1C3T5B
Kijken naar i: 5
de datum 2020-01-06 00:00:00 voor persoon 5
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 20
De lijstvrije uren ziet er als uit: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 5 8
Voltooid S1C3T5B
S1C2T3A met: False
Aanroep functie InplannenOpdracht met dagindex: 0
Er wordt ene kopie gemaakt van dfKalender
De crewlijst die wordt uitgeprobeerd: [4, 20, 20, 20, 20, 20]
bovenaan de while loop met: S1C2T3A
Kijken naar i: 4
De 

Voltooid S1C2T2B
S1C1T1B met: False
Aanroep functie InplannenOpdracht met dagindex: 0
Er wordt ene kopie gemaakt van dfKalender
De crewlijst die wordt uitgeprobeerd: [9, 1, 0, 20, 20, 20]
bovenaan de while loop met: S1C1T1B
Kijken naar i: 9
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T1B
Kijken naar i: 9
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T1B
Kijken naar i: 9
de datum 2020-01-08 00:00:00 voor persoon 9
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 1
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Kijken naar i: 20
De lijstvrije uren ziet er als uit: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 5 8
ingeplandS1C1T1B 3
bovenaan de while loop met: S1C1T1B
Kijken naar i: 11
de datum 2020-01-09 00:00:00 voor persoon 11
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 10
de datum 2020-01-09 00:00:00 voor persoon 10
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 0
de datum 2020-01-09 00:00:00 voor persoon 0
Het aantal uren te gaan voor persoon i 3.0
Kijken naar i: 20
De lijstvrije uren ziet er als uit: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 9 12
Voltooid S1C1T1B
S1C2T3B met: False
Aanroep functie InplannenOpdracht met dagindex: 0
Er wordt ene kopie gemaakt van dfKalender
De crewlijst die wordt uitgeprobeerd: [4, 0, 20, 20, 20, 20]
bovenaan de while loop met: S1C2T3B
Kijken naar i: 4
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet e

De crewlijst die wordt uitgeprobeerd: [5, 0, 20, 20, 20, 20]
bovenaan de while loop met: S1C1T2
Kijken naar i: 5
De vergeleken waardes: 0.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T2
Kijken naar i: 5
de datum 2020-01-07 00:00:00 voor persoon 5
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T2
Kijken naar i: 5
de datum 2020-01-08 00:00:00 voor persoon 5
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd n

Kijken naar i: 8
de datum 2020-01-09 00:00:00 voor persoon 8
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 4
bovenaan de while loop met: S1C1T3
Kijken naar i: 8
de datum 2020-01-10 00:00:00 voor persoon 8
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
de datum 2020-01-10 00:00:00 voor persoon 0
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 20
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 1 4
ingeplandS1C1T3 3
bovenaan de while loop met: S1C1T3
Kijken naar i: 8
de datum 2020-01-10 00:00:00 voor persoon 8
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 0
de datum 2020-01-10 00:00:00 voor persoon 0
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 20
De lijstvrije uren ziet e

...

dagindex wordt verhoogd naar: 5
Een kopie wordt teruggezet van dfKalender
De crewlijst die wordt uitgeprobeerd: [10, 9, 1, 20, 20, 20]
bovenaan de while loop met: S1C1T4
Kijken naar i: 10
De vergeleken waardes: 0.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T4
Kijken naar i: 10
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T4
Kijken naar i: 10
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 3
bovenaan de while loop met: S1C1T4
Kijken naar i: 10
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te ga

De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 5
Een kopie wordt teruggezet van dfKalender
De crewlijst die wordt uitgeprobeerd: [9, 7, 0, 20, 20, 20]
bovenaan de while loop met: S1C1T4
Kijken naar i: 9
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T4
Kijken naar i: 9
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T4
Kijken naar i: 9
de datum 2020-01-08 00:00:00 voor persoon 9
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 7
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0,

de datum 2020-01-06 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 7
de datum 2020-01-06 00:00:00 voor persoon 7
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
De vergeleken waardes: 0.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-07 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 7
de datum 2020-01-07 00:00:00 voor persoon 7
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 0
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-08 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken

de datum 2020-01-06 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 10
De vergeleken waardes: 0.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 1
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-07 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 10
De vergeleken waardes: 1.0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 2
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-08 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 10
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar

Kijken naar i: 12
de datum 2020-01-08 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 7
De vergeleken waardes: 0 2.0
De controle van het aantal uur te gaan is False
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dagindex wordt verhoogd naar: 3
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-09 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 7
de datum 2020-01-09 00:00:00 voor persoon 7
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 6
de datum 2020-01-09 00:00:00 voor persoon 6
Het aantal uren te gaan voor persoon i 7
Kijken naar i: 20
De lijstvrije uren ziet er als uit: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
het begin en het einde van de rij: 1 4
ingeplandS1C1T4 6
bovenaan de while loop met: S1C1T4
Kijken naar i: 12
de datum 2020-01-09 00:00:00 voor persoon 12
Het aantal uren te gaan voor persoon i 5.0
Kijken naar i: 7
de datum 2020-01-09 00:0

In [43]:
# dfKalender = copy.deepcopy(dfk)
# dfSprint1 = copy.deepcopy(dfs)

In [44]:
tasktest = dfSprint1[dfSprint1['Task'] == 'S1C2T3A']

!TODO mogelijkheden sorteren op mogelijkheden, niet op kamers. > techA, TechB in plaats van TechA,TechB

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? JA

In [45]:
dfSprint1

,Task,ReqSkill 1,ReqSkill 2,ReqSkill 3,ReqSkill 4,ReqSkill 5,ReqSkill 6,ReqSkill 7,ReqSkill 8,ReqSkill 9,...,MaxReqCrew,Frequency,Duration,Comments,Crew,Mogelijkheden,AantalMogelijkheden,Voltooid,Eis,Moment Voltooid
0,S1C3T5B,0,1,0,0,0,0,1,0.0,0.0,...,2.0,2.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,56,True,S1C3T5A,
1,S1C2T3A,0,0,0,0,0,0,1,0.0,0.0,...,2.0,2.0,1.5,0,20,<__main__.MogelijkeCombinaties object at 0x000...,110,True,no,
2,S1C3T5A,0,1,0,0,0,0,0,0.0,0.0,...,2.0,2.0,1.5,0,20,<__main__.MogelijkeCombinaties object at 0x000...,180,True,no,
3,S1C2T2A,0,0,0,1,0,0,0,0.0,0.0,...,2.0,4.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,230,True,no,
4,S1C3T4,0,0,0,0,0,0,0,0.0,0.0,...,2.0,1.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,275,True,no,
5,S1C3T1,0,1,1,1,1,0,0,0.0,0.0,...,4.0,1.0,3.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,1860,True,no,
6,S1C2T1,0,0,0,1,0,0,0,0.0,0.0,...,5.0,1.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,3240,True,no,
7,S1C3T2B,0,0,0,1,1,0,1,0.0,0.0,...,6.0,1.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,5816,True,S1C3T2A,
8,S1C2T2B,0,0,0,1,0,0,1,0.0,0.0,...,6.0,2.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,6780,True,S1C2T2A,
9,S1C1T1B,0,0,0,1,0,0,1,0.0,0.0,...,6.0,4.0,2.0,0,20,<__main__.MogelijkeCombinaties object at 0x000...,6780,True,S1C1T1A,


In [46]:
dfKalender.iloc[0,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,4,4,0,0,5,7,7,1,0,3,7
1,09:00:00,S1C3T4 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 1 kamer,0,0,0,S1C2T2A freq: 1 kamer,S1C3T1 freq: 1 kamer,S1C3T2A freq: 1 kamer,0
2,09:30:00,S1C3T4 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 1 kamer,0,0,0,S1C2T2A freq: 1 kamer,S1C3T1 freq: 1 kamer,S1C3T2A freq: 1 kamer,0
3,10:00:00,S1C3T4 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 1 kamer,0,0,0,S1C2T2A freq: 1 kamer,S1C3T1 freq: 1 kamer,S1C3T2A freq: 1 kamer,0
4,10:30:00,S1C3T4 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 1 kamer,0,0,0,S1C2T2A freq: 1 kamer,S1C3T1 freq: 1 kamer,S1C3T2A freq: 1 kamer,0
5,11:00:00,S1C2T5 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 2 kamer,0,0,0,S1C2T2A freq: 2 kamer,S1C3T1 freq: 1 kamer,0,0
6,11:30:00,S1C2T5 freq: 1 kamer,0,S1C3T1 freq: 1 kamer,S1C3T1 freq: 1 kamer,0,S1C3T5B freq: 2 kamer,0,0,0,S1C2T2A freq: 2 kamer,S1C3T1 freq: 1 kamer,0,0
7,12:30:00,S1C2T4 freq: 1 kamer,0,0,0,0,S1C3T5B freq: 2 kamer,0,0,0,S1C2T2A freq: 2 kamer,S1C3T2B freq: 1 kamer,S1C3T2B freq: 1 kamer,0
8,13:00:00,S1C2T4 freq: 1 kamer,0,0,0,0,S1C3T5B freq: 2 kamer,0,0,0,S1C2T2A freq: 2 kamer,S1C3T2B freq: 1 kamer,S1C3T2B freq: 1 kamer,0
9,13:30:00,S1C2T4 freq: 1 kamer,0,0,0,0,S1C3T5A freq: 1 kamer,0,0,0,S1C2T2A freq: 3 kamer,S1C3T2B freq: 1 kamer,S1C3T2B freq: 1 kamer,0


In [47]:
dfKalender.iloc[1,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,5,7,5,2,3,7,7,7,1,1,7,7
1,09:00:00,S1C1T5 freq: 1 kamer,0,0,0,S1C2T3A freq: 1 kamer,S1C1T2 freq: 1 kamer,0,0,0,S1C2T2A freq: 4 kamer,S1C1T2 freq: 1 kamer,0,0
2,09:30:00,S1C1T5 freq: 1 kamer,0,0,0,S1C2T3A freq: 1 kamer,S1C1T2 freq: 1 kamer,0,0,0,S1C2T2A freq: 4 kamer,S1C1T2 freq: 1 kamer,0,0
3,10:00:00,0,0,0,0,S1C2T3A freq: 1 kamer,S1C1T2 freq: 1 kamer,0,0,0,S1C2T2A freq: 4 kamer,S1C1T2 freq: 1 kamer,0,0
4,10:30:00,0,0,0,0,S1C2T3A freq: 2 kamer,S1C1T2 freq: 1 kamer,0,0,0,S1C2T2A freq: 4 kamer,S1C1T2 freq: 1 kamer,0,0
5,11:00:00,S1C2T1 freq: 1 kamer,0,0,S1C2T1 freq: 1 kamer,S1C2T3A freq: 2 kamer,0,0,0,0,S1C2T1 freq: 1 kamer,S1C2T1 freq: 1 kamer,0,0
6,11:30:00,S1C2T1 freq: 1 kamer,0,0,S1C2T1 freq: 1 kamer,S1C2T3A freq: 2 kamer,0,0,0,0,S1C2T1 freq: 1 kamer,S1C2T1 freq: 1 kamer,0,0
7,12:30:00,S1C2T1 freq: 1 kamer,0,0,S1C2T1 freq: 1 kamer,0,0,0,0,0,S1C2T1 freq: 1 kamer,S1C2T1 freq: 1 kamer,0,0
8,13:00:00,S1C2T1 freq: 1 kamer,0,0,S1C2T1 freq: 1 kamer,0,0,0,0,0,S1C2T1 freq: 1 kamer,S1C2T1 freq: 1 kamer,0,0
9,13:30:00,S1C2T2B freq: 1 kamer,S1C2T2B freq: 1 kamer,0,0,0,S1C1T2 freq: 2 kamer,0,0,0,S1C2T2B freq: 1 kamer,S1C1T2 freq: 2 kamer,0,0


In [48]:
dfKalender.iloc[2,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,0,7,0,7,7,0,0,7,7,0,7,7
1,09:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
2,09:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
5,11:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
6,11:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
7,12:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0
8,13:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0
9,13:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
dfKalender.iloc[3,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,0,1,7,7,7,1,7,7,7,1,1,7,7
1,09:00:00,S1C2T2B freq: 2 kamer,S1C2T2B freq: 2 kamer,0,0,0,S1C1T2 freq: 3 kamer,0,0,0,S1C2T2B freq: 2 kamer,S1C1T2 freq: 3 kamer,0,0
2,09:30:00,S1C2T2B freq: 2 kamer,S1C2T2B freq: 2 kamer,0,0,0,S1C1T2 freq: 3 kamer,0,0,0,S1C2T2B freq: 2 kamer,S1C1T2 freq: 3 kamer,0,0
3,10:00:00,S1C2T2B freq: 2 kamer,S1C2T2B freq: 2 kamer,0,0,0,S1C1T2 freq: 3 kamer,0,0,0,S1C2T2B freq: 2 kamer,S1C1T2 freq: 3 kamer,0,0
4,10:30:00,S1C2T2B freq: 2 kamer,S1C2T2B freq: 2 kamer,0,0,0,S1C1T2 freq: 3 kamer,0,0,0,S1C2T2B freq: 2 kamer,S1C1T2 freq: 3 kamer,0,0
5,11:00:00,S1C1T1B freq: 1 kamer,S1C1T1B freq: 1 kamer,0,0,0,S1C1T2 freq: 4 kamer,0,0,0,S1C1T1B freq: 1 kamer,S1C1T2 freq: 4 kamer,0,0
6,11:30:00,S1C1T1B freq: 1 kamer,S1C1T1B freq: 1 kamer,0,0,0,S1C1T2 freq: 4 kamer,0,0,0,S1C1T1B freq: 1 kamer,S1C1T2 freq: 4 kamer,0,0
7,12:30:00,S1C1T1B freq: 1 kamer,S1C1T1B freq: 1 kamer,0,0,0,S1C1T2 freq: 4 kamer,0,0,0,S1C1T1B freq: 1 kamer,S1C1T2 freq: 4 kamer,0,0
8,13:00:00,S1C1T1B freq: 1 kamer,S1C1T1B freq: 1 kamer,0,0,0,S1C1T2 freq: 4 kamer,0,0,0,S1C1T1B freq: 1 kamer,S1C1T2 freq: 4 kamer,0,0
9,13:30:00,S1C1T1B freq: 2 kamer,S1C1T1B freq: 2 kamer,0,0,0,S1C1T2 freq: 5 kamer,0,0,0,S1C1T1B freq: 2 kamer,S1C1T2 freq: 5 kamer,0,0


In [50]:
dfKalender.iloc[4,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston
0,urenTeGaan,3,0,7,0,0,7,7,7,7,0,0,7,7
1,09:00:00,S1C2T4 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
2,09:30:00,S1C2T4 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
3,10:00:00,S1C2T4 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
4,10:30:00,S1C2T4 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
5,11:00:00,S1C1T5 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
6,11:30:00,S1C1T5 freq: 3 kamer,0,0,0,0,0,0,0,0,0,0,0,0
7,12:30:00,S1C1T5 freq: 4 kamer,0,0,0,0,0,0,0,0,0,0,0,0
8,13:00:00,S1C1T5 freq: 4 kamer,0,0,0,0,0,0,0,0,0,0,0,0
9,13:30:00,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# import json
# with open('data.json', 'w', encoding='utf-8') as f:
#     json.dump(dfSprint1, f, ensure_ascii=False, indent=4)

# lijst = []
# for i in range(0,len(dfSprint1)):
#     a = dfSprint1.iloc[i,:].Mogelijkheden.dfMogelijkheden.to_json()
#     lijst.append(a)
    
# with open('df.json', 'w', encoding='utf-8') as f:
#     json.dump(lijst, f, ensure_ascii=False, indent=4)